In [1]:
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin
import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
class PerfeccionarTexto(BaseEstimator, TransformerMixin):
    def __init__(self, by=1, columns=None):
        self.by = by
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        print(type(X))
        cols_to_transform = list(X.columns)
        if self.columns:
            cols_to_transform = self.columns
        X[cols_to_transform] = X[cols_to_transform].apply(self.remove_non_ascii_text) # OJO: Respetar el estandar de python para POO, uso del self
        X[cols_to_transform] = X[cols_to_transform].apply(self.contract) 
        X[cols_to_transform] = X[cols_to_transform].apply(self.tokenize)
        X[cols_to_transform] = X[cols_to_transform].apply(self.to_lowercase)
        X[cols_to_transform] = X[cols_to_transform].apply(self.remove_punctuation)
        X[cols_to_transform] = X[cols_to_transform].apply(self.replace_numbers)
        X[cols_to_transform] = X[cols_to_transform].apply(self.remove_stopwords)
        X[cols_to_transform] = X[cols_to_transform].apply(self.stem_and_lemmatize)
        X[cols_to_transform] = X[cols_to_transform].apply(self.transformarString)
        X[cols_to_transform] = X[cols_to_transform].apply(self.vectorizar)
        
        
        return X
    
    
    def remove_non_ascii_text(self, text):
        """Remove non-ASCII characters from text"""
        new_words = ''
        serie=[]
        for word in text:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words = new_words + new_word
            serie.append(new_words)
            new_words = ''
        return serie

    def to_lowercase(self,texto):
        """Convert all characters to lowercase from list of tokenized words"""
        if texto is None:
            return
        if len(texto) == 0:
            return
        new_words = []
        serie=[]
        # text = words[1:-1].split(',')
        for words in texto:
            for word in words:
                new_word = str(word).lower()
                new_words.append(new_word)
            serie.append(new_words)
            new_words = []
        return serie

    def remove_punctuation(self,text):
        """Remove punctuation from list of tokenized words"""
        if text is None:
            return
        if len(text) == 0:
            return
        new_words = []
        serie=[]
        for words in text:
            for word in words:
                new_word = re.sub(r'[^\w\s]', '', word)
                if new_word != '':
                    new_words.append(new_word)
                    
            serie.append(new_words)
            new_words = []
        
        return serie

    def replace_numbers(self,text):
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        if text is None:
            return
        if len(text) == 0:
            return
        p = inflect.engine()
        new_words = []
        serie=[]
        for words in text:
            for word in words:
                if word.isdigit() and int(word) < 100000:
                    new_word = p.number_to_words(word)
                    new_words.append(new_word)
                else:
                    new_words.append(word)
            serie.append(new_words)
            new_words = []
        return serie

    def remove_stopwords(self,text):
        """Remove stop words from list of tokenized words"""
        if text is None:
            return
        if len(text) == 0:
            return
        new_words = []
        serie=[]
        for words in text:
            for word in words:
                if word not in stopwords.words('english'):
                    new_words.append(word)
            serie.append(new_words)
            new_words = []

        return serie
    
    def stem_words(self,words):
        """Stem words in list of tokenized words"""
        
        if words is None:
            return
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        
        return stems

    def lemmatize_verbs(self,words):
        """Lemmatize verbs in list of tokenized words"""
        if words is None:
            return
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        
        return lemmas

    def stem_and_lemmatize(self,text):
        
        serie=[]
        for words in text:
            stems = self.stem_words(words)
            lemmas = self.lemmatize_verbs(words)
            
            serie.append(stems + lemmas)

            stems=[]
            lemmas=[]
        
        if text is None:
            return
        if len(text) == 0:
            return
        
        return serie
    
    def contract(self,textos):
        if textos is None:
            return
        expanded_words = [] 
        serie=[]
        for texto in textos:
            # using contractions.fix to expand the shortened words
            expanded_words.append(contractions.fix(texto))  

            expanded_text = ' '.join(expanded_words)
            serie.append(expanded_text)
            expanded_words = []
        
        return serie
    
    def tokenize(self,textos):
        if textos is None:
            return
        tokens=[]
        serie=[]
        for texto in textos:
            
            tokens=word_tokenize(texto)
            serie.append(tokens)
            tokens=[]
        return serie
    
    def transformarString(self,textos):
        serie=[]
        escrito=""
        for words in textos:
            escrito=' '.join([str(x) for x in words])
            serie.append(escrito)
            escrito=""
        
        return serie
    
    def vectorizar(self , textos):
        count = CountVectorizer()
        return count.fit_transform(textos)
        



In [3]:
data_n=pd.read_csv('SuicidiosProyecto.csv', sep=',', encoding = 'utf-8')
data=data_n.head(5)

In [4]:
data['class'] = data['class'].replace({'suicide': 1, 'non-suicide': 0})

In [5]:
data.head()

,Unnamed: 0,text,class
0,173271,i want to destroy myselffor once everything wa...,1
1,336321,I kinda got behind schedule with learning for ...,0
2,256637,I'm just not sure anymoreFirst and foremost: I...,1
3,303772,please give me a reason to liveThats too much ...,1
4,293747,27f struggling to find meaning moving forwardI...,1


In [6]:
# Como solo se debe enviar la columna de texto, no hay necesidad de usar las clases de Column Transformer
pipeline = Pipeline(steps=[('t', PerfeccionarTexto()), ('m', DecisionTreeClassifier())])

In [7]:
# Aquí por facilidad se decide enviar solo la columna de texto, pero como dataframe dado que el transformer de ustedes no trabaja solo con Series
pipeline = pipeline.fit(data[['text']], data['class'])

<class 'pandas.core.frame.DataFrame'>


TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [ ]:


# Usamos la lbreria joblib
filename = r'C:\Users\marti\OneDrive\Documentos\Sexto semestre\BI\proyecto1\BI-Labs\Proyecto1-2\modeloText.joblib'
# Se guarda
dump(pipeline, filename) 

